In [103]:
import sys, os, importlib
import rasterio
#import reverse_geocode

import geopandas as gpd
import pandas as pd

# Import GOST libraries; sys.path.append will be unnecessary if libraries are already installed

sys.path.append(r"C:\repos\GOST_Urban")

sys.path.append(r"C:\repos\gostrocks\src")

import GOSTRocks.rasterMisc as rMisc
#from GOSTRocks.misc import tPrint
import src.UrbanRaster as urban
%matplotlib inline  

In [104]:
# Extract NUTS divisions
#nuts_file = "/home/public/Data/GLOBAL/ADMIN/NUTS/NUTS_RG_01M_2021_4326.geojson"
nuts_file = r"C:\Users\war-machine\Documents\world_bank_work\turkey_urban_analysis\DATA\ref-nuts-2021-01\NUTS_RG_01M_2021_4326.geojson"

inN = gpd.read_file(nuts_file)

inT1 = inN.loc[(inN['CNTR_CODE'] == "TR") & (inN['LEVL_CODE'] == 1)]
inT2 = inN.loc[(inN['CNTR_CODE'] == "TR") & (inN['LEVL_CODE'] == 2)]
inT3 = inN.loc[(inN['CNTR_CODE'] == "TR") & (inN['LEVL_CODE'] == 3)]


In [105]:
inT1

,id,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,FID,geometry
1284,TR8,TR8,1,TR,Batı Karadeniz,Batı Karadeniz,0.0,NaN,NaN,TR8,"POLYGON ((35.51370 41.63598, 35.56717 41.63112..."
1322,TR1,TR1,1,TR,İstanbul,İstanbul,0.0,NaN,NaN,TR1,"MULTIPOLYGON (((28.18166 41.56406, 28.18987 41..."
1323,TR2,TR2,1,TR,Batı Marmara,Batı Marmara,0.0,NaN,NaN,TR2,"MULTIPOLYGON (((28.16478 40.39561, 28.16460 40..."
1344,TRA,TRA,1,TR,Kuzeydoğu Anadolu,Kuzeydoğu Anadolu,0.0,NaN,NaN,TRA,"POLYGON ((43.46964 41.05734, 43.48579 41.02216..."
1374,TR9,TR9,1,TR,Doğu Karadeniz,Doğu Karadeniz,0.0,NaN,NaN,TR9,"POLYGON ((42.51517 41.43828, 42.52663 41.40842..."
1437,TRC,TRC,1,TR,Güneydoğu Anadolu,Güneydoğu Anadolu,0.0,NaN,NaN,TRC,"POLYGON ((41.38055 38.49225, 41.43313 38.50464..."
1567,TR3,TR3,1,TR,Ege,Ege,0.0,NaN,NaN,TR3,"MULTIPOLYGON (((29.74218 39.87133, 29.73424 39..."
1580,TR5,TR5,1,TR,Batı Anadolu,Batı Anadolu,0.0,NaN,NaN,TR5,"POLYGON ((32.55482 40.69194, 32.59134 40.68434..."
1596,TR6,TR6,1,TR,Akdeniz,Akdeniz,0.0,NaN,NaN,TR6,"MULTIPOLYGON (((37.63867 37.93117, 37.64377 37..."
1672,TR4,TR4,1,TR,Doğu Marmara,Doğu Marmara,0.0,NaN,NaN,TR4,"MULTIPOLYGON (((30.35469 41.18419, 30.40936 41..."


In [106]:
# add area
# https://gis.stackexchange.com/questions/218450/getting-polygon-areas-using-geopandas
# use mollweide

In [107]:
inT1 = inT1.to_crs("esri:54009")
inT1["area_sq_km"] = inT1['geometry'].area / 10**6
inT1 = inT1.to_crs("epsg:4326")
inT1.head(2)

,id,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,FID,geometry,area_sq_km
1284,TR8,TR8,1,TR,Batı Karadeniz,Batı Karadeniz,0.0,NaN,NaN,TR8,"POLYGON ((35.51370 41.63598, 35.56717 41.63112...",74109.820628
1322,TR1,TR1,1,TR,İstanbul,İstanbul,0.0,NaN,NaN,TR1,"MULTIPOLYGON (((28.18166 41.56406, 28.18987 41...",5345.075814


In [108]:
inT2 = inT2.to_crs("esri:54009")
inT2["area_sq_km"] = inT2['geometry'].area / 10**6
inT2 = inT2.to_crs("epsg:4326")
inT2.head(2)

,id,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,FID,geometry,area_sq_km
1294,TR71,TR71,2,TR,"Kırıkkale, Aksaray, Niğde, Nevşehir, Kırşehir","Kırıkkale, Aksaray, Niğde, Nevşehir, Kırşehir",0.0,NaN,NaN,TR71,"POLYGON ((34.17551 39.94043, 34.13163 39.92938...",31911.17722
1295,TRC1,TRC1,2,TR,"Gaziantep, Adıyaman, Kilis","Gaziantep, Adıyaman, Kilis",0.0,NaN,NaN,TRC1,"POLYGON ((39.12275 38.18331, 39.14349 38.18228...",15763.15385


In [109]:
inT3 = inT3.to_crs("esri:54009")
inT3["area_sq_km"] = inT3['geometry'].area / 10**6
inT3 = inT3.to_crs("epsg:4326")
inT3.head(2)

,id,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,FID,geometry,area_sq_km
1443,TR613,TR613,3,TR,Burdur,Burdur,3.0,2.0,3.0,TR613,"POLYGON ((29.71326 36.95631, 29.69055 36.96739...",7148.068617
1479,TR521,TR521,3,TR,Konya,Konya,4.0,2.0,3.0,TR521,"POLYGON ((34.03357 38.00481, 34.08921 38.00199...",40744.487464


# Summarize nighttime lights

In [110]:
def summarize_dmsp(inD, clip = False, out_folder = r"C:\Users\war-machine\Documents\world_bank_work\turkey_urban_analysis\output"):
    ''' Zonal stats for all VIIRS and DMSP data'''
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
    
    print("**** run zonal stats on DMSP nighttime lights")
    #ntl_folder = "/home/public/Data/GLOBAL/NighttimeLights/DMSP"
    ntl_folder = r"D:\Google_drive\GOST_Work\DMSP"
    dmsp_tiffs = os.listdir(ntl_folder)
    dmsp_res = {}
    
    for d_tiff in dmsp_tiffs:        
        inR = rasterio.open(os.path.join(ntl_folder, d_tiff))
        
        # This will end up clipping the first NTL time period and the last NTL time period
        if clip and ("F101992" in d_tiff or "F189012" in d_tiff):
            out_tiff = os.path.join(out_folder, d_tiff)
            print("Clipping %s" % out_tiff)
            if not os.path.exists(out_tiff):
                rMisc.clipRaster(inR, inD, out_tiff)
                
        title = d_tiff.split(".")[0]
        res = rMisc.zonalStats(inD, inR, minVal=0)
        res = pd.DataFrame(res, columns=["%s_%s" % (title, x) for x in ['SUM','MIN','MAX','MEAN']])
        dmsp_res[title] = res
    
    for key, vals in dmsp_res.items():
        xx = vals.iloc[:,[0,3]]
        try:        
            final = final.join(xx)
        except:
            final = xx
       
    return(final)

In [111]:
def summarize_viirs(inD, clip = False, out_folder = r"C:\Users\war-machine\Documents\world_bank_work\turkey_urban_analysis\output"):
    ''' Zonal stats for all VIIRS and DMSP data'''
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
        
    #viirs_folder = "/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED"
    viirs_folder = r"D:\Google_drive\GOST_Work\VIIRS_CORRECTED"
    
    viirs_vrts = []
    for root, dirs, files in os.walk(viirs_folder):
        for f in files:
            if f[-4:] == ".vrt":
                viirs_vrts.append(os.path.join(root, f))
                
    viirs_vrts.sort()
                
    print("Run zonal stats on VIIRS")
    all_res = {}
    for vrt in viirs_vrts:
        print(vrt)
        yr = os.path.basename(vrt).split("_")[0]
        res = rMisc.zonalStats(inD, vrt, minVal=0.5)
        res = pd.DataFrame(res, columns=["YR_%s_%s" % (yr, x) for x in ['SUM','MIN','MAX','MEAN']])
        all_res[yr] = res
        
        # This will end up clipping the first NTL time period and the last NTL time period
        if clip and (yr == '2012' or yr == "2019"):
            out_file = os.path.join(out_folder, vrt.replace(".vrt", ".tif"))
            if not os.path.exists(out_file):
                rMisc.clipRaster(rasterio.open(vrt), inD, out_file)
    
    for key, vals in all_res.items():
        xx = vals.iloc[:,[0,3]]
        try:        
            final = final.join(xx)
        except:
            final = xx
    
    return(final)

### testing

In [112]:
# summarize night-time lights for NUTS level 1
#final = summarize_dmsp(inT1, clip=False)

In [113]:
#final

In [114]:
#inT1

In [115]:
#inT1_reset = inT1.reset_index()

In [116]:
#inT1_reset

In [117]:
#final2 = inT1_reset.join(final)

In [118]:
#final2

In [119]:
# sum_cols = [col for col in final2.columns if 'SUM' in col]
# print(list(final2.columns))
# print(sum_cols)

In [120]:
# This generates columns that divide the sum of each nuts area by the area in sq km
# for col in sum_cols:
#     final2[col[:-3]+'NTL_per_sq_km'] = final2[col] / final2['area']

In [121]:
#final2

In [122]:
# NTL_per_sq_km_cols = [col for col in final2.columns if 'NTL_per_sq_km' in col]
# print(list(final2.columns))
# print(NTL_per_sq_km_cols)

In [123]:
# This generates columns that have an index value compared to the first annual year value
# for col in NTL_per_sq_km_cols:
#     final2[col[:-3]+'NTL_per_sq_km_index'] = 100 * (final2[col] / final2['F101992_NTL_per_sq_km'])

In [124]:
#final2

## Calculate states for DMSP years

In [129]:
for inT in [inT1, inT2, inT3]:    
    final = summarize_dmsp(inT)
    
    inT_reset_index = inT.reset_index()
    
    final = inT_reset_index.join(final)
    
    sum_cols = [col for col in final.columns if 'SUM' in col]
    for col in sum_cols:
        final[col[:-3]+'NTL_per_sq_km'] = final[col] / final['area_sq_km']
    
    NTL_per_sq_km_cols = [col for col in final.columns if 'NTL_per_sq_km' in col]
    for col in NTL_per_sq_km_cols:
        final[col+'_index'] = 100 * (final[col] / final['F101992_NTL_per_sq_km'])
    
    final.astype({'geometry': str}).to_csv(r"C:\Users\war-machine\Documents\world_bank_work\turkey_urban_analysis\output\NUTS%s_DMSP.csv" % inT.iloc[0,2])

**** run zonal stats on DMSP nighttime lights
**** run zonal stats on DMSP nighttime lights
**** run zonal stats on DMSP nighttime lights


## Calculate states for VIIRS years

In [130]:
for inT in [inT1, inT2, inT3]:    
    final = summarize_viirs(inT)
    
    inT_reset_index = inT.reset_index()
    
    final = inT_reset_index.join(final)
    
    sum_cols = [col for col in final.columns if 'SUM' in col]
    for col in sum_cols:
        final[col[:-3]+'NTL_per_sq_km'] = final[col] / final['area_sq_km']
    
    NTL_per_sq_km_cols = [col for col in final.columns if 'NTL_per_sq_km' in col]
    for col in NTL_per_sq_km_cols:
        final[col+'_index'] = 100 * (final[col] / final['YR_2012_NTL_per_sq_km'])
        
    final.to_csv(r"C:\Users\war-machine\Documents\world_bank_work\turkey_urban_analysis\output\NUTS%s_VIIRS.csv" % inT.iloc[0,2])

Run zonal stats on VIIRS
D:\Google_drive\GOST_Work\VIIRS_CORRECTED\2012_VIIRS_annual_composite.vrt
D:\Google_drive\GOST_Work\VIIRS_CORRECTED\2013_VIIRS_annual_composite.vrt
D:\Google_drive\GOST_Work\VIIRS_CORRECTED\2014_VIIRS_annual_composite.vrt
D:\Google_drive\GOST_Work\VIIRS_CORRECTED\2015_VIIRS_annual_composite.vrt
D:\Google_drive\GOST_Work\VIIRS_CORRECTED\2016_VIIRS_annual_composite.vrt
D:\Google_drive\GOST_Work\VIIRS_CORRECTED\2017_VIIRS_annual_composite.vrt
D:\Google_drive\GOST_Work\VIIRS_CORRECTED\2018_VIIRS_annual_composite.vrt
D:\Google_drive\GOST_Work\VIIRS_CORRECTED\2019_VIIRS_annual_composite.vrt
Run zonal stats on VIIRS
D:\Google_drive\GOST_Work\VIIRS_CORRECTED\2012_VIIRS_annual_composite.vrt
D:\Google_drive\GOST_Work\VIIRS_CORRECTED\2013_VIIRS_annual_composite.vrt
D:\Google_drive\GOST_Work\VIIRS_CORRECTED\2014_VIIRS_annual_composite.vrt
D:\Google_drive\GOST_Work\VIIRS_CORRECTED\2015_VIIRS_annual_composite.vrt
D:\Google_drive\GOST_Work\VIIRS_CORRECTED\2016_VIIRS_annual_co

In [127]:
#final.to_csv("/home/wb411133/temp/TUR/NUTS2_VIIRS.csv")

In [128]:
# testing

# final = summarize_viirs(inT1)
    
# inT_reset_index = inT1.reset_index()

# final = inT_reset_index.join(final)

# final